# Deep Learning 2025/12/03：圖神經網路基本教學
- 教學目標：
  - 了解圖神經網路 (Graph Neural Networks, GNN) 的基本概念
  - 掌握圖神經網路的主要架構與運作原理
  - 實作簡單的圖神經網路模型
- 函式庫：
  - [PyTorch Geometric](https://pytorch-geometric.readthedocs.io/en/latest/)
- 實作模型：GCN (Graph Convolutional Network)

In [ ]:
!pip install torch_geometric

In [1]:
# 0. 導入所需套件

import torch
from torch import Tensor # for typing hint
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import Planetoid
from tqdm import tqdm

## 資料集：Cora
Cora 是一個常用的圖神經網路資料集：
- 包含 **2708** 篇科學論文，這些論文被分類為七個不同的主題類別。
- 每篇論文被表示為一個節點，節點之間的引用關係則表示為邊。
- 每個節點有一個 **1433** 維的特徵向量，表示該論文的詞袋模型 (Bag-of-Words) 表示法。
- 任務為節點分類 (Node Classification)。

In [2]:
# 1. 載入資料集

"""
Planetoid: The citation network datasets "Cora", "CiteSeer" and "PubMed" from
the Revisiting Semi-Supervised Learning with Graph Embeddings” paper.
"""
dataset = Planetoid(root='.', name='Cora')

In [ ]:
# 2. 探索資料集內容

print(f'Dataset: {dataset}:')
print('====================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of nodes (節點數量): {dataset[0].num_nodes}')
print(f'Number of edges (邊的數量): {dataset[0].num_edges}')
print(f'Edge index shape (邊的索引矩陣形狀): {dataset[0].edge_index.shape}')
print(f'Number of features (向量長度): {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

Dataset: Cora():
Number of graphs: 1
Number of nodes (節點數量): 2708
Number of edges (邊的數量): 10556
Edge index shape (邊的索引矩陣形狀): torch.Size([2, 10556])
Number of features (向量長度): 1433
Number of classes: 7


In [11]:
# 3. 查看節點特徵向量的內容與形狀

print(f"特徵向量的內容: {dataset[0].x[0]}")
print(f"特徵向量的形狀: {dataset[0].x[0].size()}")

特徵向量的內容: tensor([0., 0., 0.,  ..., 0., 0., 0.])
特徵向量的形狀: torch.Size([1433])


In [33]:
# 4. 查看圖的邊的索引矩陣內容

print(dataset[0].edge_index.shape)
print(dataset[0].edge_index)

torch.Size([2, 10556])
tensor([[ 633, 1862, 2582,  ...,  598, 1473, 2706],
        [   0,    0,    0,  ..., 2707, 2707, 2707]])


In [ ]:
# 5. (optional) 測試將邊的索引矩陣轉為稀疏鄰接矩陣 (sparse adjacency matrix)

N = dataset[0].num_nodes
A = torch.sparse_coo_tensor(
    indices=dataset[0].edge_index,
    values=torch.ones(dataset[0].edge_index.shape[1]),
    size=(N, N)
)
print(A.shape) # COO 的 size 只是告訴你 dense 後會是幾維，但它本身不存整個矩陣
print(A.to_dense())  # 將稀疏矩陣轉為密集矩陣並列印

torch.Size([2708, 2708])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.],
        [0., 0., 0.,  ..., 0., 1., 0.]])


### 資料集觀察心得
1. Cora 只有一張圖 (single graph)。
2. 每個節點的特徵向量是稀疏的 (sparse)，大部分的值都是零。
3. 資料集有一個屬性叫做 `edge_index`，它是一個形狀為 `[2, num_edges]` 的矩陣，用來表示圖中的邊。
4. 每 1 條無向邊會被當成 2 條有向邊，因此 `num_edges` 是實際邊數的兩倍。
5. `edge_index` 是 COO（Coordinate List）格式，第一列表示起始節點，第二列表示目標節點。COO 的好處有：
    - 節省記憶體空間，因為只存儲非零元素的索引。
    - 適合用於稀疏圖，因為大部分的節點對之間沒有邊。
    - 實作上便於訪問相鄰節點，因此易於進行圖卷積操作。

## 模型訓練

In [ ]:
# 6. 訓練前準備

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = dataset[0].to(device)

In [ ]:
# 7. 定義 GCN 模型

class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x: Tensor, edge_index: Tensor) -> Tensor:
        # x: Node feature matrix of shape [num_nodes, in_channels]
        # edge_index: Graph connectivity matrix of shape [2, num_edges]
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x

In [ ]:
# 8. 建立模型實例

model = GCN(
    in_channels=dataset.num_features,
    hidden_channels=16,
    out_channels=dataset.num_classes,
).to(device)

In [ ]:
# 9. 模型訓練

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

pbar = tqdm(range(200), desc="Training")
for epoch in pbar:
    pred = model(data.x, data.edge_index)
    loss = F.cross_entropy(pred[data.train_mask], data.y[data.train_mask])

    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # 更新進度條右側資訊
    pbar.set_postfix(loss=f"{loss.item():.4f}")

Training: 100%|██████████| 200/200 [00:00<00:00, 330.16it/s, loss=0.0007]


In [ ]:
# 10. 模型評估

model.eval()
pred = model(data.x, data.edge_index).argmax(dim=1)

correct = int((pred[data.test_mask] == data.y[data.test_mask]).sum())
acc = correct / int(data.test_mask.sum())
print(f'Test Accuracy: {acc:.4f}')

Test Accuracy: 0.7760
